# 03 - Gold Layer (Camada Ouro)
## Métricas de Negócio

Criar métricas essenciais para análise.


In [1]:
import pandas as pd
import numpy as np
import os


## 1. Carregar Dados


In [ ]:
# Carregar dados limpos
df = pd.read_csv('data/silver/dados_limpos.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['data_compra'] = df['InvoiceDate'].dt.date

print(f"Dados carregados: {len(df):,} registros")
df.head()


✓ Dados carregados: 536,639 registros


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,data_ingestao,fonte_arquivo,eh_devolucao,valor_total,data_processamento,data_compra
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,15.30,2025-10-31 19:40:08.366383,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,20.34,2025-10-31 19:40:08.366383,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,22.00,2025-10-31 19:40:08.366383,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,20.34,2025-10-31 19:40:08.366383,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,20.34,2025-10-31 19:40:08.366383,2010-12-01


## 2. Preparar Diretórios


In [3]:
os.makedirs('data/gold', exist_ok=True)


## 3. Métricas Diárias


In [ ]:
# Métricas por dia
metricas_diarias = df.groupby('data_compra').agg({
    'InvoiceNo': 'nunique',
    'valor_total': 'sum',
    'CustomerID': 'nunique'
}).reset_index()

metricas_diarias.columns = ['data', 'total_pedidos', 'receita_total', 'clientes_unicos']
metricas_diarias['ticket_medio'] = (metricas_diarias['receita_total'] / metricas_diarias['total_pedidos']).round(2)
metricas_diarias = metricas_diarias.sort_values('data')

metricas_diarias.to_csv('data/gold/metricas_diarias.csv', index=False)

print(f" Métricas diárias criadas: {len(metricas_diarias)} dias")
metricas_diarias.tail()


✓ Métricas diárias criadas: 305 dias


,data,total_pedidos,receita_total,clientes_unicos,ticket_medio
300,2011-12-05,151,57630.20,115,381.66
301,2011-12-06,149,54109.39,129,363.15
302,2011-12-07,138,74952.61,102,543.13
303,2011-12-08,145,81294.33,121,560.65
304,2011-12-09,49,32111.91,39,655.35


## 4. Análise RFM de Clientes


In [ ]:
# Análise RFM por cliente
data_referencia = df['InvoiceDate'].max()

analise_clientes = df.groupby('CustomerID').agg({
    'InvoiceDate': 'max',
    'InvoiceNo': 'nunique',
    'valor_total': 'sum'
}).reset_index()

analise_clientes.columns = ['id_cliente', 'ultima_compra', 'frequencia_pedidos', 'valor_total_gasto']
analise_clientes['dias_ultima_compra'] = (data_referencia - analise_clientes['ultima_compra']).dt.days

def calcular_score_rfm(serie, labels):
    """Calcula score RFM com tratamento de valores duplicados"""
    try:
        result = pd.qcut(serie, q=5, labels=labels, duplicates='drop')
        return result
    except ValueError:
        try:
            _, bins = pd.qcut(serie, q=5, duplicates='drop', retbins=True)
            n_bins = len(bins) - 1
            return pd.qcut(serie, q=n_bins, labels=labels[:n_bins], duplicates='drop')
        except ValueError:
           
            return pd.cut(serie, bins=5, labels=labels, duplicates='drop')

# Aplicar scores
analise_clientes['score_recencia'] = calcular_score_rfm(analise_clientes['dias_ultima_compra'], [5,4,3,2,1])
analise_clientes['score_frequencia'] = calcular_score_rfm(analise_clientes['frequencia_pedidos'], [1,2,3,4,5])
analise_clientes['score_monetario'] = calcular_score_rfm(analise_clientes['valor_total_gasto'], [1,2,3,4,5])

analise_clientes['score_rfm_total'] = analise_clientes[['score_recencia', 'score_frequencia', 'score_monetario']].astype(float).sum(axis=1)

analise_clientes.to_csv('data/gold/analise_clientes.csv', index=False)

print(f"Análise de clientes criada: {len(analise_clientes)} clientes")
analise_clientes.head()


✓ Análise de clientes criada: 4373 clientes


,id_cliente,ultima_compra,frequencia_pedidos,valor_total_gasto,dias_ultima_compra,score_recencia,score_frequencia,score_monetario,score_rfm_total
0,0,2011-12-09 10:26:00,3708,1469611.65,0,5,5,5,15.0
1,12346,2011-01-18 10:17:00,2,0.00,325,1,1,1,3.0
2,12347,2011-12-07 15:52:00,7,4310.00,1,5,1,5,11.0
3,12348,2011-09-25 13:13:00,4,1797.24,74,2,1,4,7.0
4,12349,2011-11-21 09:51:00,1,1757.55,18,4,1,4,9.0


## 5. Desempenho de Produtos


In [ ]:
# Desempenho por produto
desempenho_produtos = df.groupby(['StockCode', 'Description']).agg({
    'Quantity': 'sum',
    'valor_total': 'sum',
    'InvoiceNo': 'nunique'
}).reset_index()

desempenho_produtos.columns = ['codigo_produto', 'nome_produto', 'quantidade_vendida', 'receita_total', 'num_pedidos']
desempenho_produtos = desempenho_produtos.sort_values('receita_total', ascending=False)

desempenho_produtos.to_csv('data/gold/desempenho_produtos.csv', index=False)

print(f"Desempenho de produtos criado: {len(desempenho_produtos)} produtos")
desempenho_produtos.head()


✓ Desempenho de produtos criado: 5747 produtos


,codigo_produto,nome_produto,quantidade_vendida,receita_total,num_pedidos
5731,DOT,DOTCOM POSTAGE,707,206245.48,709
1932,22423,REGENCY CAKESTAND 3 TIER,13007,164459.49,2169
3872,47566,PARTY BUNTING,18010,98243.88,1706
5095,85123A,WHITE HANGING HEART T-LIGHT HOLDER,35006,97659.94,2235
5063,85099B,JUMBO BAG RED RETROSPOT,47260,92175.79,2135


## 6. Vendas por País


In [ ]:
# Vendas por país
analise_paises = df.groupby('Country').agg({
    'InvoiceNo': 'nunique',
    'valor_total': 'sum',
    'CustomerID': 'nunique'
}).reset_index()

analise_paises.columns = ['pais', 'total_pedidos', 'receita_total', 'clientes_unicos']
analise_paises = analise_paises.sort_values('receita_total', ascending=False)

analise_paises.to_csv('data/gold/analise_paises.csv', index=False)

print(f"Análise de países criada: {len(analise_paises)} países")
analise_paises.head()


✓ Análise de países criada: 38 países


,pais,total_pedidos,receita_total,clientes_unicos
36,United Kingdom,23492,8189252.304,3951
24,Netherlands,101,284661.540,9
10,EIRE,360,262993.380,4
14,Germany,603,221509.470,95
13,France,461,197317.110,88


## 7. Métricas de Devoluções


In [ ]:
# Análise de devoluções
devolucoes = df[df['eh_devolucao'] == True]

if len(devolucoes) > 0:
    analise_devolucoes = devolucoes.groupby(['StockCode', 'Description']).agg({
        'Quantity': 'sum',
        'valor_total': 'sum',
        'InvoiceNo': 'nunique'
    }).reset_index()
    
    analise_devolucoes.columns = ['codigo_produto', 'nome_produto', 'quantidade_devolvida', 'valor_devolvido', 'num_devolucoes']
    analise_devolucoes = analise_devolucoes.sort_values('valor_devolvido', ascending=True)
    
    analise_devolucoes.to_csv('data/gold/analise_devolucoes.csv', index=False)
    
    print(f"Análise de devoluções criada: {len(analise_devolucoes)} produtos")
    analise_devolucoes.head()
else:
    print("Não há devoluções nos dados")


✓ Análise de devoluções criada: 3181 produtos


## 8. Métricas Mensais


In [ ]:
# Extrair ano e mês
df['ano_mes'] = df['InvoiceDate'].dt.to_period('M').astype(str)

# Agregar por mês
metricas_mensais = df.groupby('ano_mes').agg({
    'InvoiceNo': 'nunique',
    'valor_total': 'sum',
    'CustomerID': 'nunique'
}).reset_index()

metricas_mensais.columns = ['ano_mes', 'total_pedidos', 'receita_total', 'clientes_unicos']
metricas_mensais['ticket_medio'] = (metricas_mensais['receita_total'] / metricas_mensais['total_pedidos']).round(2)

# Salvar
metricas_mensais.to_csv('data/gold/metricas_mensais.csv', index=False)

print(f"Métricas mensais criadas: {len(metricas_mensais)} meses")
metricas_mensais.tail()


✓ Métricas mensais criadas: 13 meses


,ano_mes,total_pedidos,receita_total,clientes_unicos,ticket_medio
8,2011-08,1735,703510.580,981,405.48
9,2011-09,2327,1017596.682,1303,437.30
10,2011-10,2637,1069368.230,1426,405.52
11,2011-11,3462,1456145.800,1712,420.61
12,2011-12,1015,432701.060,687,426.31
